In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.special as special 
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Cargar archivo csv des seaborn

df= pd.read_csv('Datos_Limpios_Athenas_NNNA.csv')
df

,Unnamed: 0,id,host_id,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,...,last_scraped,source,host_response_time,host_is_superhost,host_verifications,neighbourhood,property_type,room_type,amenities,license
0,0,27262,37177,100.0,100.0,4.0,12.0,37.989240,23.700000,2.0,...,2024-12-25,city scrape,within an hour,t,"['email', 'phone']",Dato desconocido,Entire rental unit,Entire home/apt,"[""Heating - split type ductless system"", ""Dish...",00002433111
1,1,809874,4259738,100.0,96.0,2.0,2.0,37.962150,23.721790,4.0,...,2024-12-25,city scrape,within an hour,f,"['email', 'phone']",Dato desconocido,Entire rental unit,Entire home/apt,"[""Host greets you"", ""Dishes and silverware"", ""...",00001240742
2,2,866381,4551671,100.0,97.0,1.0,1.0,38.000000,23.724430,3.0,...,2024-12-25,city scrape,within a few hours,t,"['email', 'phone', 'work_email']",Dato desconocido,Entire condo,Entire home/apt,"[""Coffee"", ""Dishes and silverware"", ""Washer"", ...",00002608390
3,3,886724,4700824,100.0,99.6,1.0,1.0,37.997450,23.739730,2.0,...,2024-12-25,city scrape,Dato desconocido,f,['email'],"Athens, Attica, Greece",Entire rental unit,Entire home/apt,"[""Host greets you"", ""Luggage dropoff allowed"",...",00000052101
4,4,896212,4777984,100.0,100.0,29.0,50.0,37.988440,23.738450,2.0,...,2024-12-25,city scrape,within an hour,t,"['email', 'phone']","Athens, Attica, Greece",Entire rental unit,Entire home/apt,"[""Host greets you"", ""Coffee"", ""Dishes and silv...",00000206136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14637,14637,1318251942056598277,25811452,100.0,100.0,3.0,3.0,37.979336,23.726354,4.0,...,2024-12-26,city scrape,within an hour,t,"['email', 'phone']",Dato desconocido,Entire rental unit,Entire home/apt,"[""Coffee"", ""Dishes and silverware"", ""Washer"", ...",00003108542
14638,14638,1318276742106864141,666320836,100.0,99.6,2.0,2.0,37.992753,23.700000,2.0,...,2024-12-25,city scrape,within a few hours,f,['phone'],Dato desconocido,Entire condo,Entire home/apt,"[""Heating - split type ductless system"", ""Coff...",00002888548
14639,14639,1318307768558324736,221534605,100.0,98.0,53.0,12.5,37.983118,23.756604,2.0,...,2024-12-25,city scrape,within a few hours,f,"['email', 'phone', 'work_email']","Athina, Greece",Entire rental unit,Entire home/apt,"[""TV"", ""Hair dryer"", ""Refrigerator"", ""Washer \...",00002764099
14640,14640,1318405788507138933,650411545,100.0,99.6,1.0,1.0,37.984651,23.735136,4.0,...,2024-12-25,city scrape,Dato desconocido,f,"['email', 'phone']",Dato desconocido,Entire rental unit,Entire home/apt,"[""Coffee"", ""Outlet covers"", ""Washer"", ""Dishes ...",00003072289


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14642 entries, 0 to 14641
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Unnamed: 0                                    14642 non-null  int64  
 1   id                                            14642 non-null  int64  
 2   host_id                                       14642 non-null  int64  
 3   host_response_rate                            14642 non-null  float64
 4   host_acceptance_rate                          14642 non-null  float64
 5   host_listings_count                           14642 non-null  float64
 6   host_total_listings_count                     14642 non-null  float64
 7   latitude                                      14642 non-null  float64
 8   longitude                                     14642 non-null  float64
 9   accommodates                                  14642 non-null 

### Cambio de categorias 

**Primer cambio**

In [4]:
#verificar los valores sin repetirse de una columna
unico = np.unique(df['host_response_time'])
print(unico)

#convertimos una variable categorica  dicotomica
df['host_response_time']= df['host_response_time'].replace(['Dato desconocido', 'within a few hours', 'within an hour'], 'Less than a day')
df['host_response_time']= df['host_response_time'].replace(['a few days or more', 'within a day'], 'More than a day')
df

['Dato desconocido' 'a few days or more' 'within a day'
 'within a few hours' 'within an hour']


,Unnamed: 0,id,host_id,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,...,last_scraped,source,host_response_time,host_is_superhost,host_verifications,neighbourhood,property_type,room_type,amenities,license
0,0,27262,37177,100.0,100.0,4.0,12.0,37.989240,23.700000,2.0,...,2024-12-25,city scrape,Less than a day,t,"['email', 'phone']",Dato desconocido,Entire rental unit,Entire home/apt,"[""Heating - split type ductless system"", ""Dish...",00002433111
1,1,809874,4259738,100.0,96.0,2.0,2.0,37.962150,23.721790,4.0,...,2024-12-25,city scrape,Less than a day,f,"['email', 'phone']",Dato desconocido,Entire rental unit,Entire home/apt,"[""Host greets you"", ""Dishes and silverware"", ""...",00001240742
2,2,866381,4551671,100.0,97.0,1.0,1.0,38.000000,23.724430,3.0,...,2024-12-25,city scrape,Less than a day,t,"['email', 'phone', 'work_email']",Dato desconocido,Entire condo,Entire home/apt,"[""Coffee"", ""Dishes and silverware"", ""Washer"", ...",00002608390
3,3,886724,4700824,100.0,99.6,1.0,1.0,37.997450,23.739730,2.0,...,2024-12-25,city scrape,Less than a day,f,['email'],"Athens, Attica, Greece",Entire rental unit,Entire home/apt,"[""Host greets you"", ""Luggage dropoff allowed"",...",00000052101
4,4,896212,4777984,100.0,100.0,29.0,50.0,37.988440,23.738450,2.0,...,2024-12-25,city scrape,Less than a day,t,"['email', 'phone']","Athens, Attica, Greece",Entire rental unit,Entire home/apt,"[""Host greets you"", ""Coffee"", ""Dishes and silv...",00000206136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14637,14637,1318251942056598277,25811452,100.0,100.0,3.0,3.0,37.979336,23.726354,4.0,...,2024-12-26,city scrape,Less than a day,t,"['email', 'phone']",Dato desconocido,Entire rental unit,Entire home/apt,"[""Coffee"", ""Dishes and silverware"", ""Washer"", ...",00003108542
14638,14638,1318276742106864141,666320836,100.0,99.6,2.0,2.0,37.992753,23.700000,2.0,...,2024-12-25,city scrape,Less than a day,f,['phone'],Dato desconocido,Entire condo,Entire home/apt,"[""Heating - split type ductless system"", ""Coff...",00002888548
14639,14639,1318307768558324736,221534605,100.0,98.0,53.0,12.5,37.983118,23.756604,2.0,...,2024-12-25,city scrape,Less than a day,f,"['email', 'phone', 'work_email']","Athina, Greece",Entire rental unit,Entire home/apt,"[""TV"", ""Hair dryer"", ""Refrigerator"", ""Washer \...",00002764099
14640,14640,1318405788507138933,650411545,100.0,99.6,1.0,1.0,37.984651,23.735136,4.0,...,2024-12-25,city scrape,Less than a day,f,"['email', 'phone']",Dato desconocido,Entire rental unit,Entire home/apt,"[""Coffee"", ""Outlet covers"", ""Washer"", ""Dishes ...",00003072289


### Primer caso

In [6]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['minimum_minimum_nights', 'minimum_nights', 'minimum_nights_avg_ntm']]
vars_Dep=df[['host_is_superhost']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='t')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='t')
print('sensibilidad del modelo: ')
print(sensibilidad)

Matriz de confusión: 
[[2550   72]
 [1705   66]]

precision del modelo: 
0.4782608695652174

exactitud del modelo
0.5954928295014796

sensibilidad del modelo: 
0.037267080745341616


c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Segundo caso

In [7]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['accommodates', 'review_scores_rating', 'bedrooms']]
vars_Dep=df[['host_response_time']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='More than a day')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='More than a day')
print('sensibilidad del modelo: ')
print(sensibilidad)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Matriz de confusión: 
[[4228    0]
 [ 165    0]]

precision del modelo: 
0.0

exactitud del modelo
0.9624402458456636

sensibilidad del modelo: 
0.0


### Tercer caso

In [10]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['minimum_nights_avg_ntm', 'maximum_minimum_nights', 'host_listings_count']]
vars_Dep=df[['source']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='previous scrape')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='previous scrape')
print('sensibilidad del modelo: ')
print(sensibilidad)

Matriz de confusión: 
[[4105    0]
 [ 288    0]]

precision del modelo: 
0.0

exactitud del modelo
0.9344411563851582

sensibilidad del modelo: 
0.0


c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Cuarto caso

In [11]:
#Obtenmos el limite superiro y el limite inferior de la columna objetivo

max = df['review_scores_rating'].max()
min=df['review_scores_rating'].min()
limites = [min,max]
print(limites)

#Categorización de varibales
#declaramos 2 intervalos

intervalos=np.linspace(4.64, 5.1,3)
print(intervalos)

#Creamos las categorias
categorias = ['Bad score', 'Good score']

#Finalmente creamos las categorias en la columa numerica
df['review_scores_rating'] = pd.cut(x=df['review_scores_rating'], bins = intervalos, labels= categorias)



[np.float64(4.65), np.float64(5.0)]
[4.64 4.87 5.1 ]


In [12]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['minimum_nights_avg_ntm', 'maximum_minimum_nights', 'host_listings_count']]
vars_Dep=df[['review_scores_rating']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='Bad score')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='Bad score')
print('sensibilidad del modelo: ')
print(sensibilidad)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Matriz de confusión: 
[[   0 1082]
 [   0 3311]]

precision del modelo: 
0.0

exactitud del modelo
0.753699066697018

sensibilidad del modelo: 
0.0


### Quinto caso

In [13]:
#Obtenmos el limite superiro y el limite inferior de la columna objetivo

max = df['availability_90'].max()
min=df['availability_90'].min()
limites = [min,max]
print(limites)

#Categorización de varibales
#declaramos 2 intervalos

intervalos=np.linspace(34.9, 90.1,3)
print(intervalos)

#Creamos las categorias
categorias = ['Baja demanda', 'Alta demanda']

#Finalmente creamos las categorias en la columa numerica
df['availability_90'] = pd.cut(x=df['availability_90'], bins = intervalos, labels= categorias)

[np.float64(35.0), np.float64(90.0)]
[34.9 62.5 90.1]


In [14]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['minimum_nights_avg_ntm', 'maximum_minimum_nights', 'host_listings_count']]
vars_Dep=df[['availability_90']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='Baja demanda')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='Baja demanda')
print('sensibilidad del modelo: ')
print(sensibilidad)

Matriz de confusión: 
[[3727    0]
 [ 666    0]]

precision del modelo: 
0.0

exactitud del modelo
0.8483951741406783

sensibilidad del modelo: 
0.0


c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Sexto caso

In [15]:
#Obtenmos el limite superiro y el limite inferior de la columna objetivo

max = df['review_scores_cleanliness'].max()
min=df['review_scores_cleanliness'].min()
limites = [min,max]
print(limites)

#Categorización de varibales
#declaramos 2 intervalos

intervalos=np.linspace(4.65, 5.1,3)
print(intervalos)

#Creamos las categorias
categorias = ['Pocas reseñas', 'Muchas reseñas']

#Finalmente creamos las categorias en la columa numerica
df['review_scores_cleanliness'] = pd.cut(x=df['review_scores_cleanliness'], bins = intervalos, labels= categorias)

[np.float64(4.66), np.float64(5.0)]
[4.65  4.875 5.1  ]


In [16]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['reviews_per_month', 'review_scores_checkin', 'number_of_reviews']]
vars_Dep=df[['review_scores_cleanliness']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='Pocas reseñas')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='Pocas reseñas')
print('sensibilidad del modelo: ')
print(sensibilidad)

Matriz de confusión: 
[[3256   94]
 [ 943  100]]

precision del modelo: 
0.5154639175257731

exactitud del modelo
0.763942636011837

sensibilidad del modelo: 
0.09587727708533078


c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Séptimo caso

In [17]:
#Obtenmos el limite superiro y el limite inferior de la columna objetivo

max = df['price'].max()
min=df['price'].min()
limites = [min,max]
print(limites)

#Categorización de varibales
#declaramos 2 intervalos

intervalos=np.linspace(24.9, 175.1,3)
print(intervalos)

#Creamos las categorias
categorias = ['Precios bajos', 'Precios altos']

#Finalmente creamos las categorias en la columa numerica
df['price'] = pd.cut(x=df['price'], bins = intervalos, labels= categorias)

[np.float64(25.0), np.float64(175.0)]
[ 24.9 100.  175.1]


In [18]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['availability_30', 'maximum_minimum_nights', 'host_listings_count']]
vars_Dep=df[['price']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='Precios altos')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='Precios altos')
print('sensibilidad del modelo: ')
print(sensibilidad)

Matriz de confusión: 
[[   0  713]
 [   0 3680]]

precision del modelo: 
0.0

exactitud del modelo
0.837696335078534

sensibilidad del modelo: 
0.0


c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Octavo caso

In [19]:
#Obtenmos el limite superiro y el limite inferior de la columna objetivo

max = df['beds'].max()
min=df['beds'].min()
limites = [min,max]
print(limites)

#Categorización de varibales
#declaramos 2 intervalos

intervalos=np.linspace(0.9, 3.1,3)
print(intervalos)

#Creamos las categorias
categorias = ['1 cama', 'más de 2 camas']

#Finalmente creamos las categorias en la columa numerica
df['beds'] = pd.cut(x=df['beds'], bins = intervalos, labels= categorias)

[np.float64(1.0), np.float64(3.0)]
[0.9 2.  3.1]


In [21]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['availability_30', 'maximum_minimum_nights', 'host_listings_count']]
vars_Dep=df[['beds']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='más de 2 camas')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='más de 2 camas')
print('sensibilidad del modelo: ')
print(sensibilidad)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Matriz de confusión: 
[[3781    0]
 [ 612    0]]

precision del modelo: 
0.0

exactitud del modelo
0.8606874573184612

sensibilidad del modelo: 
0.0


c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


###  Noveno caso

In [22]:
#Obtenmos el limite superiro y el limite inferior de la columna objetivo

max = df['host_acceptance_rate'].max()
min=df['host_acceptance_rate'].min()
limites = [min,max]
print(limites)

#Categorización de varibales
#declaramos 2 intervalos

intervalos=np.linspace(95.9, 100.1,3)
print(intervalos)

#Creamos las categorias
categorias = ['Media alta aceptación', 'Alta aceptación']

#Finalmente creamos las categorias en la columa numerica
df['host_acceptance_rate'] = pd.cut(x=df['host_acceptance_rate'], bins = intervalos, labels= categorias)

[np.float64(96.0), np.float64(100.0)]
[ 95.9  98.  100.1]


In [23]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['availability_30', 'maximum_minimum_nights', 'host_listings_count']]
vars_Dep=df[['host_acceptance_rate']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='Media alta aceptación')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='Media alta aceptación')
print('sensibilidad del modelo: ')
print(sensibilidad)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Matriz de confusión: 
[[4031    0]
 [ 362    0]]

precision del modelo: 
0.0

exactitud del modelo
0.9175961757341224

sensibilidad del modelo: 
0.0


### Décimo caso

In [24]:
#Obtenmos el limite superiro y el limite inferior de la columna objetivo

max = df['calculated_host_listings_count'].max()
min=df['calculated_host_listings_count'].min()
limites = [min,max]
print(limites)

#Categorización de varibales
#declaramos 2 intervalos

intervalos=np.linspace(0.9, 38.1,3)
print(intervalos)

#Creamos las categorias
categorias = ['Pocas propiedades', 'Varias propiedades']

#Finalmente creamos las categorias en la columa numerica
df['calculated_host_listings_count'] = pd.cut(x=df['calculated_host_listings_count'], bins = intervalos, labels= categorias)


[np.float64(1.0), np.float64(38.0)]
[ 0.9 19.5 38.1]


In [25]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=df[['availability_30', 'maximum_minimum_nights', 'host_listings_count']]
vars_Dep=df[['calculated_host_listings_count']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label ='Varias propiedades')
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label='Varias propiedades')
print('sensibilidad del modelo: ')
print(sensibilidad)

Matriz de confusión: 
[[3820  169]
 [ 312   92]]

precision del modelo: 
0.3524904214559387

exactitud del modelo
0.8905076257682677

sensibilidad del modelo: 
0.22772277227722773


c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
